In [1]:
import pandas as pd
import requests
import json
import glob
import os
import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

## GET AUTHORISATION TOKEN
Grab your token and your client ID used to request an authorisation token (https://supercoach.heraldsun.com.au/2020/api/afl/classic/v1/access_token) and input them into the payload

In [2]:
# Start the session
session = requests.Session()

# Create the payload
payload = {
    "token": "eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCIsImtpZCI6Ik1UVkdNRGxETUVaRk1qWkdOVU0yT0RnNFF6VTROalZCTlRsR1FrUTRSamM1TVROQ09UWTJOZyJ9.eyJodHRwOi8vbG9naW4ubmV3c2NvcnBhdXN0cmFsaWEuY29tLmF1L3Byb2ZpbGUiOnsicmF0IjoxNTU0ODg1OTk2LCJ0aGlua19pZCI6IjE0MTIzMjMxIiwiY3VzdG9tZXJfcHJvZHVjdF9ob2xkaW5nIjpbIlpaX05MX1JFRyIsIlpaX1NDX0REQUZMIiwiWlpfU0NfRERGUk5PRU5UIl0sInNpdGUiOiJzdXBlcmNvYWNoX2hlcmFsZHN1biIsImhhc19jb25zZW50ZWRfdG9fdGMiOnRydWUsImF0c19oYXNoIjoiZjZmMjQ2NGUwNGFiNzgzYWJjN2FkYTA3ZDIwZTZmYjMwNmExOWRiNjYzMjVmYjY4ZWRhOGQ1ODNiYThkNTk4ZS41NmNmOTgyMzcyMmM1MDEwYzE5YjdiOTIwMWZmOGZjZTUzNjc2N2ExIiwiYXV0aFByb3ZpZGVyIjoiYXV0aDAifSwibmlja25hbWUiOiJNYXJrIiwibmFtZSI6Ik1hcmsiLCJwaWN0dXJlIjoiaHR0cHM6Ly9ncmFwaC5mYWNlYm9vay5jb20vNzE1NjE3NzU2L3BpY3R1cmUiLCJ1cGRhdGVkX2F0IjoiMjAyMS0wOC0xN1QwMjoxNjoyMy43ODVaIiwiaXNzIjoiaHR0cHM6Ly9sb2dpbi5uZXdzY29ycGF1c3RyYWxpYS5jb20vIiwic3ViIjoiYXV0aDB8YzVjZWVjOWItYWQ4NC00NWE5LWFlNWMtOTBiNjc1YzRiMWMxIiwiYXVkIjoiWllDb3RsaWhxYUd1YXFTc1N2dTBMMnZ4RGRRWEN3MTYiLCJpYXQiOjE2MjkxNjY1ODMsImV4cCI6MTYzNjUxMDU4MywiYXRfaGFzaCI6ImlCSDJnODBka24xLVFHUHp3bXpKUmciLCJub25jZSI6IjVxZUN2b0tIcjVzTUkzSkFrdktjY0szaDRIVEpzMFNWIn0.t-dOSYj_2rZ_YL9dKOc7UB5dOdRizpVJDVGCZoiZKfHp-l0ib5sLRnz9h1JvvZlTVmUPWtFtaxWjlUwn5u_lWcdlw-thnCNxEOyikRisA2CKOJN1V948AFdZNbk-4n1S7PoiQsHgBSmJIu1ZjnMFRLNoDrWUiT3PwdtWPS5hZzNlyg4HLFhzL-iJzlmsHHXSKC4VHHA4FGSL_MfqwKS3JRlkhHj7WPIYnVUxMxMgHp6G21Jak89RTxu5obER6PDS9ffVJjPIf-nM2NsxL9ejuS9sBoqLS0TzWmgmaHGABoicJs257ddq0S4bEjDVkb1yoiFWLgyyWM0q58zFcg9Iug",
    "client_id": "Lf7m0371XCbMBlQ0fAFoGRJlfCs2JZpYvLU1uEvd",
    "client_secret": "",
    "grant_type": "social",
    "service": "auth0",
}


# Post the payload to the site to log in
s = session.post("https://supercoach.heraldsun.com.au/2021/api/afl/classic/v1/access_token", data=payload, verify=False)
display(s.text)
access_token = json.loads(s.text)['access_token']

headers = {
    "Authorization": "Bearer " + access_token
}

'{"token_type":"Bearer","expires_in":604800,"access_token":"ca079ed91c4cc34d0bd290d3fcb2be8c4dc8551d","refresh_token":"a69cb54c29e96f5f1da34a55e770e69ab2a0da31"}'

## PLAYER LIST

In [3]:
def get_position_from_json(positions):
    positions_string = ''
    for position in positions:
        if len(positions_string) > 0:
            positions_string = positions_string + ' ' + position['position']
        else:
            positions_string = positions_string + position['position']
    return positions_string

In [4]:
s = session.get("https://supercoach.heraldsun.com.au/2021/api/afl/draft/v1/players-cf?embed=positions", headers=headers)
player_json = json.loads(s.text)
df_player_list = pd.DataFrame()

for player in player_json:
    positions_string = get_position_from_json(player['positions'])
    df_line = pd.DataFrame(
        {
            'player_id': [player['id']],
            'feed_id': [player['feed_id']],
            'first_name': [player['first_name']],
            'last_name': [player['last_name']],
            'team': [player['team']['abbrev']],
            'position': [pos for pos in [player['positions']]],
            'position': positions_string
        }
    )
    df_player_list = pd.concat([df_player_list, df_line])
df_player_list

,player_id,feed_id,first_name,last_name,team,position
0,448,290528,Max,Gawn,MEL,RUC
0,74,293535,Lachie,Neale,BRL,MID
0,642,296205,Jack,Steele,STK,MID
0,466,996701,Clayton,Oliver,MEL,MID
0,666,295342,Jake,Lloyd,SYD,DEF
...,...,...,...,...,...,...
0,11,250417,Bryce,*** Gibbs,ADE,DEF
0,143,270861,Levi,*** Greenwood,COL,MID
0,428,291821,Jonathon,*** Patton,HAW,FWD
0,766,998414,Daniel,*** Venables,WCE,FWD


## Player List with Averages

In [5]:
s = session.get("https://supercoach.heraldsun.com.au/2021/api/afl/draft/v1/statspack?season=2020&round=99", headers=headers) 
player_json = json.loads(s.text)
df_player_list_averages = pd.DataFrame()

for player in player_json:
    df_line = pd.DataFrame(
        {
            'player_id': [player['player_id']],
            'total_games' : [player['total_games']],
            'total_points': [player['total_points']]
        }
    )
    df_player_list_averages = pd.concat([df_player_list_averages, df_line])
df_player_list_averages['average'] = df_player_list_averages['total_points'] / df_player_list_averages['total_games']
df_player_list_averages

,player_id,total_games,total_points,average
0,3,17,1150,67.647059
0,4,2,94,47.000000
0,6,16,1771,110.687500
0,7,1,51,51.000000
0,8,9,767,85.222222
...,...,...,...,...
0,766,0,0,NaN
0,767,9,542,60.222222
0,768,3,135,45.000000
0,770,6,566,94.333333


In [6]:
df_player_list_averages.to_csv('outputs/player_list_averages.csv')

## LADDER AND FIXTURE DATA
- also saves each json file

In [7]:
# specify round
round_number = 23
match_json_list = []

for rd in range(1, round_number + 1):
    s = session.get("https://supercoach.heraldsun.com.au/2021/api/afl/draft/v1/leagues/575/ladderAndFixtures?round={}&scores=true".format(rd), headers=headers)  
    match_json = json.loads(s.text)
    match_json_list.append(match_json)
    with open("outputs/json files/match_json_rd{}.json".format(rd), 'w') as outfile:
        json.dump(match_json, outfile, indent=4, sort_keys=True)

## COACH IDs AND NAMES

In [8]:
df_coaches = pd.DataFrame()
first_match_json = match_json_list[0]

for coach in first_match_json['ladder']:
    df_line = pd.DataFrame(
        {
            'coach_id': [coach['userTeam']['user']['id']],
            'coach_team_id': [coach['userTeam']['id']],
            'coach_first_name': [coach['userTeam']['user']['first_name']],
            'coach_team_name': [coach['userTeam']['teamname']],
        }
    )
    df_coaches = pd.concat([df_coaches, df_line])
    
df_coaches

,coach_id,coach_team_id,coach_first_name,coach_team_name
0,180650,23345,Jordan,Pink Lloyd
0,21562,2018,Mark,Surviving J.Kelly
0,10857,20323,Anthony,La MacRaena
0,71052,20240,James,MakingTheirMarcus
0,45168,6510,Luke,The Big Short
0,86078,22859,Lester,Ariana Grundy
0,300791,2016,Paul,At Witts End
0,8258,23413,Simon,The West Wingard


In [9]:
df_coaches.to_csv('inputs/coach_list.csv')

## PLAYER MATCH RESULTS

Function to get player scores for each match

In [10]:
def get_player_score_data(scores, on_field_status, coach_id, coach_team_id, coach_first_name, coach_team_name):
    df_player_results = pd.DataFrame()
    df_player_line = pd.DataFrame()
    for player in scores:
        player_id = player['player_id'],
        if on_field_status == True:
            points = player['points']
        else:
            points = player['player']['player_stats'][0]['points']
        played_position = player['position']
        df_player_line = pd.DataFrame(
            {
                'round': rd_number,
                'coach_id': coach_id,
                'coach_team_id': coach_team_id,
                'coach_first_name': coach_first_name,
                'coach_team_name': coach_team_name,
                'player_id': player_id,
                'played_position': played_position,
                'points': points,
                'on_field': on_field_status
            })
        df_player_results = pd.concat([df_player_results, df_player_line])
    return df_player_results

In [11]:
df_player_match_results = pd.DataFrame()

for match in match_json_list:
    for coach in match['ladder']:
        rd_number = coach['round']
        coach_id = coach['userTeam']['user']['id'],
        coach_team_id = coach['userTeam']['id'],
        coach_first_name = coach['userTeam']['user']['first_name'],
        coach_team_name = coach['userTeam']['teamname'],
        df_on_field_players = get_player_score_data(coach['userTeam']['scores']['scoring'], True, coach_id, coach_team_id, coach_first_name, coach_team_name)
        df_off_field_players = get_player_score_data(coach['userTeam']['scores']['nonscoring'], False, coach_id, coach_team_id, coach_first_name, coach_team_name)
        df_all_players = pd.concat([df_on_field_players, df_off_field_players])
        df_player_match_results = pd.concat([df_player_match_results, df_all_players])

df_player_match_results = df_player_match_results.merge(df_player_list, on='player_id')
df_player_match_results['round_number_alt'] = "R" + df_player_match_results['round'].astype('str')
df_player_match_results

,round,coach_id,coach_team_id,coach_first_name,coach_team_name,player_id,played_position,points,on_field,feed_id,first_name,last_name,team,position,round_number_alt
0,1,180650,23345,Jordan,Pink Lloyd,221,MID,125,True,1002232,Andrew,Brayshaw,FRE,MID,R1
1,2,180650,23345,Jordan,Pink Lloyd,221,MID,132,True,1002232,Andrew,Brayshaw,FRE,MID,R2
2,3,180650,23345,Jordan,Pink Lloyd,221,MID,66,True,1002232,Andrew,Brayshaw,FRE,MID,R3
3,4,180650,23345,Jordan,Pink Lloyd,221,MID,48,True,1002232,Andrew,Brayshaw,FRE,MID,R4
4,5,180650,23345,Jordan,Pink Lloyd,221,MID,117,True,1002232,Andrew,Brayshaw,FRE,MID,R5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4043,23,86078,22859,Lester,Ariana Grundy,62,FWD,0,False,993953,Eric,Hipwood,BRL,FWD,R23
4044,23,300791,2016,Paul,At Witts End,27,DEF,0,False,990882,Wayne,Milera,ADE,DEF,R23
4045,23,300791,2016,Paul,At Witts End,29,MID,0,False,1016082,Tariek,Newchurch,ADE,MID FWD,R23
4046,23,71052,20240,James,MakingTheirMarcus,662,MID,106,True,294036,George,Hewett,SYD,MID,R23


In [12]:
df_player_match_results.to_csv('outputs/player_match_results.csv')

## LADDER

In [13]:
df_ladder = pd.DataFrame()

for match in match_json_list:
    for coach in match['ladder']:
        df_line = pd.DataFrame(
            {
                'round': coach['round'],
                'coach_id': [coach['userTeam']['user']['id']],
                'coach_team_id': [coach['userTeam']['id']],
                'coach_first_name': [coach['userTeam']['user']['first_name']],
                'coach_team_name': [coach['userTeam']['teamname']],
                'wins': coach['wins'],
                'draws': coach['draws'],
                'losses': coach['losses'],
                'points_for': coach['points_for'],
                'points_against': coach['points_against'],
                'league_points': coach['points'],
                'position': coach['position']
            }
        )
        df_ladder = pd.concat([df_ladder, df_line])
df_ladder

,round,coach_id,coach_team_id,coach_first_name,coach_team_name,wins,draws,losses,points_for,points_against,league_points,position
0,1,180650,23345,Jordan,Pink Lloyd,1,0,0,1657,1235,4,1
0,1,21562,2018,Mark,Surviving J.Kelly,1,0,0,1657,1477,4,2
0,1,10857,20323,Anthony,La MacRaena,1,0,0,1691,1563,4,3
0,1,71052,20240,James,MakingTheirMarcus,1,0,0,1558,1470,4,4
0,1,45168,6510,Luke,The Big Short,0,0,1,1470,1558,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...
0,23,10857,20323,Anthony,La MacRaena,0,0,0,0,0,0,4
0,23,86078,22859,Lester,Ariana Grundy,0,0,0,0,0,0,5
0,23,180650,23345,Jordan,Pink Lloyd,0,0,0,0,0,0,6
0,23,21562,2018,Mark,Surviving J.Kelly,0,0,0,0,0,0,7


In [14]:
df_ladder.to_csv('outputs/ladder.csv')

## FIXTURE RESULTS

In [15]:
df_fixture_results = pd.DataFrame()
for match in match_json_list:
    for result in match['fixtures']:
        df_line = pd.DataFrame(
            {
                'round_number': [result['round']],
                'match_name': '{} vs {}'.format(result['user_team1']['user']['first_name'], result['user_team2']['user']['first_name']),
                'home_team_id': [result['user_team1']['id']],
                'home_team_name': [result['user_team1']['teamname']],
                'home_team_coach': [result['user_team1']['user']['first_name']],
                'home_team_points': [result['user_team1']['stats'][0]['points']],
                'away_team_id': [result['user_team2']['id']],
                'away_team_name': [result['user_team2']['teamname']],
                'away_team_coach': [result['user_team2']['user']['first_name']],
                'away_team_points': [result['user_team2']['stats'][0]['points']],
            }
        )
        df_fixture_results = pd.concat([df_fixture_results, df_line])
    
df_fixture_results

,round_number,match_name,home_team_id,home_team_name,home_team_coach,home_team_points,away_team_id,away_team_name,away_team_coach,away_team_points
0,1,Paul vs Mark,2016,At Witts End,Paul,1477,2018,Surviving J.Kelly,Mark,1657
0,1,Luke vs James,6510,The Big Short,Luke,1470,20240,MakingTheirMarcus,James,1558
0,1,Anthony vs Lester,20323,La MacRaena,Anthony,1691,22859,Ariana Grundy,Lester,1563
0,1,Jordan vs Simon,23345,Pink Lloyd,Jordan,1657,23413,The West Wingard,Simon,1235
0,2,Paul vs Luke,2016,At Witts End,Paul,1693,6510,The Big Short,Luke,1651
...,...,...,...,...,...,...,...,...,...,...
0,21,Luke vs Simon,6510,The Big Short,Luke,1706,23413,The West Wingard,Simon,1754
0,21,Anthony vs Jordan,20323,La MacRaena,Anthony,1612,23345,Pink Lloyd,Jordan,1673
0,22,Anthony vs James,20323,La MacRaena,Anthony,1561,20240,MakingTheirMarcus,James,1696
0,22,Paul vs Luke,2016,At Witts End,Paul,1938,6510,The Big Short,Luke,1789


In [16]:
df_fixture_results.to_csv('outputs/fixture_results.csv'.format(round_number))

### Rearrange fixture data to display by team + opposition scores
- Generate team rank for that week + opponent rank for that week

In [17]:
df_home_fixture_results = df_fixture_results
df_away_fixture_results = df_fixture_results

home_renamed_cols = {
     'home_team_id': 'team_id',
     'home_team_name': 'team_name',
     'home_team_coach': 'team_coach',
     'home_team_points': 'team_points',
     'away_team_id': 'opposition_team_id',
     'away_team_name': 'opposition_team_name',
     'away_team_coach': 'opposition_team_coach',
     'away_team_points': 'opposition_team_points'
}

away_renamed_cols = {
     'home_team_id': 'opposition_team_id',
     'home_team_name': 'opposition_team_name',
     'home_team_coach': 'opposition_team_coach',
     'home_team_points': 'opposition_team_points',
     'away_team_id': 'team_id',
     'away_team_name': 'team_name',
     'away_team_coach': 'team_coach',
     'away_team_points': 'team_points'
}

df_home_fixture_results = df_home_fixture_results.rename(columns=home_renamed_cols)
df_away_fixture_results = df_away_fixture_results.rename(columns=away_renamed_cols)
df_fixture_by_each_team = pd.concat([df_home_fixture_results, df_away_fixture_results])
df_fixture_by_each_team['Rank'] = df_fixture_by_each_team.groupby('round_number')['team_points'].rank(ascending=False)
df_fixture_by_each_team['Opponent Rank'] = df_fixture_by_each_team.groupby('round_number')['opposition_team_points'].rank(ascending=False)
df_fixture_by_each_team = df_fixture_by_each_team.sort_values(['round_number', 'Rank'])
df_fixture_by_each_team.to_csv('outputs/fixture_results_by_team.csv')
df_fixture_by_each_team

,round_number,match_name,team_id,team_name,team_coach,team_points,opposition_team_id,opposition_team_name,opposition_team_coach,opposition_team_points,Rank,Opponent Rank
0,1,Anthony vs Lester,20323,La MacRaena,Anthony,1691,22859,Ariana Grundy,Lester,1563,1.0,4.0
0,1,Jordan vs Simon,23345,Pink Lloyd,Jordan,1657,23413,The West Wingard,Simon,1235,2.5,8.0
0,1,Paul vs Mark,2018,Surviving J.Kelly,Mark,1657,2016,At Witts End,Paul,1477,2.5,6.0
0,1,Anthony vs Lester,22859,Ariana Grundy,Lester,1563,20323,La MacRaena,Anthony,1691,4.0,1.0
0,1,Luke vs James,20240,MakingTheirMarcus,James,1558,6510,The Big Short,Luke,1470,5.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...
0,22,Paul vs Luke,6510,The Big Short,Luke,1789,2016,At Witts End,Paul,1938,2.0,1.0
0,22,Anthony vs James,20240,MakingTheirMarcus,James,1696,20323,La MacRaena,Anthony,1561,3.0,4.0
0,22,Anthony vs James,20323,La MacRaena,Anthony,1561,20240,MakingTheirMarcus,James,1696,4.0,3.0
0,23,James vs Paul,2016,At Witts End,Paul,1897,20240,MakingTheirMarcus,James,1693,1.0,2.0


## CURRENTLY LISTED TEAMS

In [18]:
s = session.get("https://supercoach.heraldsun.com.au/2021/api/afl/draft/v1/leagues/575/playersStatus", headers=headers)
players_json = json.loads(s.text)
df_players = pd.DataFrame.from_dict(players_json, orient='index')
df_players = df_players.merge(df_player_list, on='player_id', how='left')
df_currently_listed_players = df_players.loc[df_players['trade_status'] == "owner"]
df_currently_listed_players = df_currently_listed_players.merge(df_coaches, left_on = 'user_team_id', right_on='coach_team_id', how='left')
df_currently_listed_players.to_csv('outputs/current_teams.csv')
df_currently_listed_players

,player_id,trade_status,user_team_id,feed_id,first_name,last_name,team,position,coach_id,coach_team_id,coach_first_name,coach_team_name
0,448,owner,6510.0,290528,Max,Gawn,MEL,RUC,45168,6510,Luke,The Big Short
1,74,owner,22859.0,293535,Lachie,Neale,BRL,MID,86078,22859,Lester,Ariana Grundy
2,642,owner,22859.0,296205,Jack,Steele,STK,MID,86078,22859,Lester,Ariana Grundy
3,466,owner,2018.0,996701,Clayton,Oliver,MEL,MID,21562,2018,Mark,Surviving J.Kelly
4,666,owner,23345.0,295342,Jake,Lloyd,SYD,DEF,180650,23345,Jordan,Pink Lloyd
...,...,...,...,...,...,...,...,...,...,...,...,...
171,755,owner,22859.0,1008282,Xavier,O'Neill,WCE,FWD MID,86078,22859,Lester,Ariana Grundy
172,782,owner,22859.0,290199,Majak,Daw,MEL,FWD,86078,22859,Lester,Ariana Grundy
173,29,owner,2016.0,1016082,Tariek,Newchurch,ADE,MID FWD,300791,2016,Paul,At Witts End
174,796,owner,2018.0,1020959,Daniel,Turner,MEL,DEF,21562,2018,Mark,Surviving J.Kelly


## CURRENT FREE AGENTS + WAIVER

In [19]:
df_available_players = df_players.loc[df_players['trade_status'] != "owner"]
df_available_players

,player_id,trade_status,user_team_id,feed_id,first_name,last_name,team,position
32,278,free_agent,NaN,281091,Hugh,Greenwood,GCS,MID
39,319,free_agent,NaN,281065,Mitch,Duncan,GEE,MID
41,436,free_agent,NaN,297566,James,Sicily,HAW,DEF
42,345,free_agent,NaN,280317,Rhys,Stanley,GEE,RUC
45,208,free_agent,NaN,291783,Dylan,Shiel,ESS,MID
...,...,...,...,...,...,...,...,...
800,11,free_agent,NaN,250417,Bryce,*** Gibbs,ADE,DEF
801,143,free_agent,NaN,270861,Levi,*** Greenwood,COL,MID
802,428,free_agent,NaN,291821,Jonathon,*** Patton,HAW,FWD
803,766,free_agent,NaN,998414,Daniel,*** Venables,WCE,FWD


In [20]:
df_available_players.to_csv('outputs/current_free_agents_waiver.csv')

# Transactions

## Waiver Picks

In [21]:
s = session.get("https://supercoach.heraldsun.com.au/2021/api/afl/draft/v1/leagues/575/processedWaivers", headers=headers)
waivers_json = json.loads(s.text)
df_waiver_transactions = pd.json_normalize(waivers_json)

wanted_columns = [
    'round',
    'user_team_id',
    'processed',
    'player_id',
    'player.feed_id',
    'player.first_name',
    'player.last_name',
    'player.team.abbrev',
    'dropped_player_id',
    'dropped_player.feed_id',
    'dropped_player.first_name',
    'dropped_player.last_name',
    'dropped_player.team.abbrev',
]

df_waiver_transactions = df_waiver_transactions.loc[:, wanted_columns]
df_waiver_transactions['source'] = "Waiver"
df_waiver_transactions

,round,user_team_id,processed,player_id,player.feed_id,player.first_name,player.last_name,player.team.abbrev,dropped_player_id,dropped_player.feed_id,dropped_player.first_name,dropped_player.last_name,dropped_player.team.abbrev,source
0,1,6510,2021-03-17 04:00:02,188,297452,Nick,Hind,ESS,263,998260,Jack,Bowes,GCS,Waiver
1,2,20240,2021-03-23 04:00:03,263,998260,Jack,Bowes,GCS,639,293846,Sebastian,Ross,STK,Waiver
2,2,2018,2021-03-23 04:00:03,581,990827,Jack,Graham,RIC,124,998215,Will,Setterfield,CAR,Waiver
3,2,20323,2021-03-23 04:00:03,402,1008550,Will,Day,HAW,616,294307,Brad,Crouch,STK,Waiver
4,2,23413,2021-03-23 04:00:03,159,298288,Darcy,Moore,COL,259,998130,Ben,Ainsworth,GCS,Waiver
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,22,20240,2021-08-11 04:00:02,757,281373,Jack,Redden,WCE,125,1001028,Jack,Silvagni,CAR,Waiver
195,23,2016,2021-08-17 04:00:02,547,997142,Peter,Ladhams,PTA,181,1005577,Sam,Draper,ESS,Waiver
196,23,20240,2021-08-17 04:00:02,400,291327,Jonathon,Ceglar,HAW,372,296324,Jesse,Hogan,GWS,Waiver
197,23,2016,2021-08-17 04:00:02,21,270938,Tom,Lynch,ADE,747,1011464,Ben,Johnson,WCE,Waiver


In [22]:
df_waiver_transactions.to_csv('outputs/waiver_transactions.csv')

## Free Agent trades

In [23]:
s = session.get("https://supercoach.heraldsun.com.au/2021/api/afl/draft/v1/leagues/575/trades", headers=headers)
free_agent_trades_json = json.loads(s.text)
df_fa_transactions = pd.json_normalize(free_agent_trades_json)

wanted_columns = [
    'round',
    'user_id',
    'action_time',
    'buy_player.id',
    'buy_player.feed_id',
    'buy_player.first_name',
    'buy_player.last_name',
    'buy_player.team.abbrev',
    'sell_player.id',
    'sell_player.feed_id',
    'sell_player.first_name',
    'sell_player.last_name',
    'sell_player.team.abbrev',
]

df_fa_transactions = df_fa_transactions.loc[:, wanted_columns]
df_fa_transactions['source'] = "Free Agent"
df_fa_transactions

,round,user_id,action_time,buy_player.id,buy_player.feed_id,buy_player.first_name,buy_player.last_name,buy_player.team.abbrev,sell_player.id,sell_player.feed_id,sell_player.first_name,sell_player.last_name,sell_player.team.abbrev,source
0,2,2016,2021-03-23 05:01:48,698,290085,Taylor,Duryea,WBD,287.0,996464,Darcy,MacPherson,GCS,Free Agent
1,2,2016,2021-03-23 05:01:48,528,992128,Ryan,Burton,PTA,547.0,997142,Peter,Ladhams,PTA,Free Agent
2,3,2016,2021-03-30 04:56:16,247,992059,Sam,Switkowski,FRE,739.0,298268,Liam,Duggan,WCE,Free Agent
3,4,2016,2021-04-07 05:02:17,120,298290,Marc,Pittonet,CAR,307.0,291975,Jarrod,Witts,GCS,Free Agent
4,4,2016,2021-04-07 05:02:17,187,290629,Dyson,Heppell,ESS,247.0,992059,Sam,Switkowski,FRE,Free Agent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,21,23413,2021-08-07 19:14:03,676,996765,Tom,Papley,SYD,413.0,296254,Jarman,Impey,HAW,Free Agent
360,21,23413,2021-08-07 01:15:47,413,296254,Jarman,Impey,HAW,678.0,290307,Dane,Rampe,SYD,Free Agent
361,21,23413,2021-08-07 01:13:59,62,993953,Eric,Hipwood,BRL,737.0,290826,Jamie,Cripps,WCE,Free Agent
362,21,23413,2021-08-05 23:58:05,231,291570,Nat,Fyfe,FRE,339.0,298419,Brandan,Parfitt,GEE,Free Agent


In [24]:
df_fa_transactions.to_csv('outputs/free_agent_transactions.csv')

## Team Trades

In [25]:
s = session.get("https://supercoach.heraldsun.com.au/2021/api/afl/draft/v1/leagues/575/teamtrades", headers=headers)
team_trades_json = json.loads(s.text)

In [26]:
df_trades_player_list = pd.DataFrame()

for trade in team_trades_json:
    df_lines = pd.json_normalize(trade['team_trade_players'])
    df_lines['round'] = trade['round']
    df_lines['user_team_id'] = trade['user_team_id']
    df_lines['processed'] = trade['action_time']
    df_lines['status'] = trade['status_id']
    df_lines['source'] = 'Trade'
    df_trades_player_list = pd.concat([df_trades_player_list,df_lines])
df_trades_player_list

,id,trade_id,player_id,trade_player_id,round,user_team_id,processed,status,source
0,56639,35932,360,478,5,23345,2021-04-12T19:36:56+10:00,2,Trade
0,56673,35955,360,478,5,23345,2021-04-12T19:55:47+10:00,7,Trade
0,65204,41234,625,96,6,6510,2021-04-21T16:06:29+10:00,2,Trade
0,96408,60377,507,322,9,6510,2021-05-15T08:36:41+10:00,3,Trade
0,103795,64806,677,583,10,23413,2021-05-21T19:10:42+10:00,7,Trade
1,103796,64806,0,187,10,23413,2021-05-21T19:10:42+10:00,7,Trade
2,103797,64806,0,58,10,23413,2021-05-21T19:10:42+10:00,7,Trade
0,114997,71477,579,487,12,2016,2021-06-05T19:30:48+10:00,7,Trade
0,115076,71532,664,124,13,2016,2021-06-06T22:35:59+10:00,7,Trade
0,115079,71534,664,120,13,2016,2021-06-06T22:35:59+10:00,10,Trade


In [27]:
df_team_one_trades_player_list = df_trades_player_list.loc[df_trades_player_list['status'] == 7, ['round', 'user_team_id', 'processed', 'player_id', 'source']]
df_team_two_trades_player_list = df_trades_player_list.loc[df_trades_player_list['status'] == 7, ['round', 'user_team_id', 'processed', 'trade_player_id', 'source']]
df_team_two_trades_player_list = df_team_two_trades_player_list.rename(columns={'trade_player_id': 'player_id'})
df_trades_by_player = pd.concat([df_team_one_trades_player_list, df_team_two_trades_player_list]).sort_values(by=['processed'])
df_trades_by_player = df_trades_by_player.merge(df_player_list[['player_id', 'feed_id', 'first_name', 'last_name', 'team']], on='player_id', how='left')
df_trades_by_player = df_trades_by_player.rename(columns={'feed_id': 'player.feed_id', 'first_name': 'player.first_name', 'last_name': 'player.last_name', 'team': 'player.team.abbrev'})
df_trades_by_player

,round,user_team_id,processed,player_id,source,player.feed_id,player.first_name,player.last_name,player.team.abbrev
0,5,23345,2021-04-12T19:55:47+10:00,360,Trade,1001396,Isaac,Cumming,GWS
1,5,23345,2021-04-12T19:55:47+10:00,478,Trade,291902,Jack,Viney,MEL
2,10,23413,2021-05-21T19:10:42+10:00,677,Trade,290778,Luke,Parker,SYD
3,10,23413,2021-05-21T19:10:42+10:00,0,Trade,NaN,NaN,NaN,NaN
4,10,23413,2021-05-21T19:10:42+10:00,0,Trade,NaN,NaN,NaN,NaN
5,10,23413,2021-05-21T19:10:42+10:00,187,Trade,290629,Dyson,Heppell,ESS
6,10,23413,2021-05-21T19:10:42+10:00,583,Trade,250312,Bachar,Houli,RIC
7,10,23413,2021-05-21T19:10:42+10:00,58,Trade,294518,Joe,Daniher,BRL
8,12,2016,2021-06-05T19:30:48+10:00,487,Trade,280921,Ben,Cunnington,NTH
9,12,2016,2021-06-05T19:30:48+10:00,579,Trade,260930,Shane,Edwards,RIC


## Combined Free Agents, Waiver, and Trades transactions list

In [28]:

columns = [
    'round',
    'user_team_id',
    'processed',
    'player_id',
    'player.feed_id',
    'player.first_name',
    'player.last_name',
    'player.team.abbrev',
    'dropped_player_id',
    'dropped_player.feed_id',
    'dropped_player.first_name',
    'dropped_player.last_name',
    'dropped_player.team.abbrev',
    'source',
]
df_waiver_transactions.columns = columns
df_fa_transactions.columns = columns

df_transactions = pd.concat([df_waiver_transactions, df_fa_transactions, df_trades_by_player])
df_transactions = df_transactions.sort_values(by=['processed'])
df_transactions

,round,user_team_id,processed,player_id,player.feed_id,player.first_name,player.last_name,player.team.abbrev,dropped_player_id,dropped_player.feed_id,dropped_player.first_name,dropped_player.last_name,dropped_player.team.abbrev,source
0,1,6510,2021-03-17 04:00:02,188,297452,Nick,Hind,ESS,263.0,998260,Jack,Bowes,GCS,Waiver
221,1,22859,2021-03-17 05:43:14,755,1008282,Xavier,O'Neill,WCE,624.0,295584,Bradley,Hill,STK,Free Agent
314,1,23413,2021-03-17 19:44:52,396,280744,Luke,Breust,HAW,557.0,993979,Sam,Powell-Pepper,PTA,Free Agent
56,1,2018,2021-03-17 20:17:32,608,992374,Dan,Butler,STK,339.0,298419,Brandan,Parfitt,GEE,Free Agent
315,1,23413,2021-03-17 20:25:07,59,293479,Cameron,Ellis-Yolmen,BRL,219.0,296371,Brett,Bewley,FRE,Free Agent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52,23,2016,2021-08-21 16:04:21,487,280921,Ben,Cunnington,NTH,763.0,280078,Luke,Shuey,WCE,Free Agent
53,23,2016,2021-08-21 19:08:42,29,1016082,Tariek,Newchurch,ADE,312.0,293845,Jeremy,Cameron,GEE,Free Agent
54,23,2016,2021-08-22 11:08:36,162,260257,Scott,Pendlebury,COL,258.0,1009256,Hayden,Young,FRE,Free Agent
170,23,20240,2021-08-22 14:39:20,6,297401,Matt,Crouch,ADE,198.0,998102,Andrew,McGrath,ESS,Free Agent


In [29]:
df_transactions.to_csv('outputs/transactions.csv')

## Add source to player match results

In [30]:
def get_transaction_status(row):
    post_transaction_list = pd.DataFrame()
    round_number = row['round']
    feed_id = row['player.feed_id']
    source = row['source']
    player_rows = df_player_match_results.loc[(df_player_match_results['round'] >= round_number) & (df_player_match_results['feed_id'] == feed_id), :]
    player_rows['source'] = source
    if len(player_rows) > 0:
        df_player_match_results.update(player_rows)

df_player_match_results['source'] = "Draft"
df_transactions.apply(lambda row: get_transaction_status(row), axis=1)
df_player_match_results

C:\Users\richardm\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,round,coach_id,coach_team_id,coach_first_name,coach_team_name,player_id,played_position,points,on_field,feed_id,first_name,last_name,team,position,round_number_alt,source
0,1.0,180650.0,23345.0,Jordan,Pink Lloyd,221.0,MID,125.0,True,1002232,Andrew,Brayshaw,FRE,MID,R1,Draft
1,2.0,180650.0,23345.0,Jordan,Pink Lloyd,221.0,MID,132.0,True,1002232,Andrew,Brayshaw,FRE,MID,R2,Draft
2,3.0,180650.0,23345.0,Jordan,Pink Lloyd,221.0,MID,66.0,True,1002232,Andrew,Brayshaw,FRE,MID,R3,Draft
3,4.0,180650.0,23345.0,Jordan,Pink Lloyd,221.0,MID,48.0,True,1002232,Andrew,Brayshaw,FRE,MID,R4,Draft
4,5.0,180650.0,23345.0,Jordan,Pink Lloyd,221.0,MID,117.0,True,1002232,Andrew,Brayshaw,FRE,MID,R5,Draft
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4043,23.0,86078.0,22859.0,Lester,Ariana Grundy,62.0,FWD,0.0,False,993953,Eric,Hipwood,BRL,FWD,R23,Free Agent
4044,23.0,300791.0,2016.0,Paul,At Witts End,27.0,DEF,0.0,False,990882,Wayne,Milera,ADE,DEF,R23,Free Agent
4045,23.0,300791.0,2016.0,Paul,At Witts End,29.0,MID,0.0,False,1016082,Tariek,Newchurch,ADE,MID FWD,R23,Free Agent
4046,23.0,71052.0,20240.0,James,MakingTheirMarcus,662.0,MID,106.0,True,294036,George,Hewett,SYD,MID,R23,Free Agent


In [31]:
df_player_match_results.to_csv('outputs/player_match_results.csv')